### Mariam Gasoyan
# Homework 5

In [1]:
import pandas as pd
from selenium import webdriver
import time
import requests
from collections import Counter
from scrapy.http import TextResponse
import re

### 1) Info on top taxpayers in provided on https://www.harkatu.am/ in a JavaScript generated table.

#### a) scrape the paid taxes, organization name and industry for top 200 taxpayers (last Quarter) .

In [2]:
url = 'https://www.harkatu.am/companies/hy?1'

In [3]:
ratings_200 = []
companies_200 = []
industry_200 = []
sales_200 = []

In [4]:
browser = webdriver.Chrome()

while len(ratings_200) < 200:
    browser.get(url)
    time.sleep(3)
    page = browser.page_source
    response = TextResponse(url=url,body=page,encoding="utf-8")
    companies = response.xpath('//div[@class="itemname"]/h1/a/span/text()').extract()
    companies_200.extend(companies)
    industry = response.xpath('//div[@class="industry"]/text()').extract()
    industry_200.extend(industry)
    ratings = response.xpath('//td[@class="raiting"]/div/text()').extract()
    ratings_200.extend(ratings)
    sales = response.xpath('//td[@class="sales-volume"]/div/text()').extract()
    sales = [sales[i*2+1] for i in range(25)]
    sales_200.extend(sales)
    next_page = browser.find_element_by_class_name('next')
    next_page.click()
    time.sleep(3)
    url = browser.current_url

browser.close()

In [5]:
print(len(ratings_200))
print(len(companies_200))
print(len(industry_200))
print(len(sales_200))

200
200
183
200


There are some companies that don't have the industry mentioned. So I will not include them in my dataframe in this case.

#### b) what is the average tax paid?

In [6]:
sales_200 = [int(x.replace(',', '')) for x in sales_200]

In [7]:
df = pd.DataFrame({'Company': companies_200, 'Rating' :ratings_200, 'Total_Taxes': sales_200})

In [8]:
df.head()

,Company,Rating,Total_Taxes
0,ԳԱԶՊՐՈՄ ԱՐՄԵՆԻԱ,1,13570605
1,ԳՐԱՆԴ ՏՈԲԱԿՈ,2,12830020
2,ԶԱՆԳԵԶՈՒՐԻ ՊՂՆՁԱՄՈԼԻԲԴԵՆԱՅԻՆ ԿՈՄԲԻՆԱՏ,3,6684445
3,ՀԱՅԱՍՏԱՆԻ ԷԼԵԿՏՐԱԿԱՆ ՑԱՆՑԵՐ,4,4544427
4,ԳԵՈՊՐՈՄԱՅՆԻՆԳ ԳՈԼԴ,5,4295157


In [9]:
print("Average tax:", df.Total_Taxes.mean(), "AMD")

Average tax: 809770.225 AMD


#### c) what is the top industry in the scraped list?

In [10]:
industry_200[30:40]

['Խմիչքների արտադրություն',
 'Ցամաքային տրանսպորտ',
 'Բանկային ոլորտ',
 'Մանրածախ առևտուր, Սննդամթերքի և խմիչքի վաճառք',
 'Ներմուծում',
 'Կահույքի վաճառք, Մանրածախ առևտուր, Տրանսպորտ, Համակարգչային տեխնիկայի ներմուծում, Էլեկտրատեխնիկայի վաճառք, Կահույքի ներմուծում, Կենցաղային տեխնիկայի ներմուծում, Մեծածախ առևտուր, Շինանյութի վաճառք, Սպորտային ապրանքների վաճառք, Համակարգչային տեխնիկայի վաճառք, Կահույքի արտադրություն, Ներմուծում',
 'Ծխախոտային արտադրատեսակների վաճառք',
 'Մանրածախ առևտուր, Մեծածախ առևտուր',
 'Բանկային ոլորտ',
 'Ծրագրային ապահովում']

Some companies belong to more than one industry. 
<br> So, I need to seperate them.

In [11]:
ind_str = ''
for i in industry_200:
    ind_str = ind_str + i + ', '

ind_list = ind_str.split(',')

ind_list = [i.strip() for i in ind_list]

len(ind_list)

267

Top industries:

In [12]:
Counter(ind_list).most_common(10)

[('Բանկային ոլորտ', 14),
 ('Մեծածախ առևտուր', 13),
 ('Արդյունաբերություն', 12),
 ('Ներմուծում', 12),
 ('Սննդամթերքի և խմիչքի վաճառք', 11),
 ('Խմիչքների արտադրություն', 11),
 ('Սննդամթերքի և խմիչքի ներմուծում', 9),
 ('Սննդամթերքի արտադրություն', 8),
 ('Ծրագրային ապահովում', 8),
 ('Մանրածախ առևտուր', 7)]

#### * It could go even this way

In [13]:
url = 'https://www.harkatu.am/companies/hy?1'

In [14]:
browser = webdriver.Chrome()
tables = []

while len(tables) < 200:
    browser.get(url)
    time.sleep(6)
    page = browser.page_source
    current_tables = pd.read_html(page)[:25]
    tables.extend(current_tables)
    next_page = browser.find_element_by_class_name('next')
    next_page.click()
    time.sleep(3)
    url = browser.current_url

browser.close()

In [15]:
df2 = pd.concat(tables)

In [16]:
df2.head(2)

,0,1,2,3,4,5,6
0,1,NaN,ԳԱԶՊՐՈՄ ԱՐՄԵՆԻԱ Գազի մատակարարում Երևան 6041,1 2,"var values = [13996698.50,6739555.53,9635324.9...","13,531,276 41.01%","13,570,605 41.2%"
0,2,NaN,ԳՐԱՆԴ ՏՈԲԱԿՈ Ծխախոտային արտադրատեսակներ Երևա...,2 1,"var values = [8070773.10,8628767.83,10996418.5...","12,126,039 10.76%","12,830,020 11.68%"


In [17]:
del(df2[1], df2[3], df2[4], df2[5])

In [18]:
df2.head()

,0,2,6
0,1,ԳԱԶՊՐՈՄ ԱՐՄԵՆԻԱ Գազի մատակարարում Երևան 6041,"13,570,605 41.2%"
0,2,ԳՐԱՆԴ ՏՈԲԱԿՈ Ծխախոտային արտադրատեսակներ Երևա...,"12,830,020 11.68%"
0,3,ԶԱՆԳԵԶՈՒՐԻ ՊՂՆՁԱՄՈԼԻԲԴԵՆԱՅԻՆ ԿՈՄԲԻՆԱՏ Մետաղակ...,"6,684,445 18.98%"
0,4,ՀԱՅԱՍՏԱՆԻ ԷԼԵԿՏՐԱԿԱՆ ՑԱՆՑԵՐ Էլեկտրականության ...,"4,544,427 106.26%"
0,5,ԳԵՈՊՐՈՄԱՅՆԻՆԳ ԳՈԼԴ Մետաղական հանքարդյունաբերո...,"4,295,157 3.61%"


In [19]:
df2[2] = [re.findall('.*[Ա-Ֆ]{2}', a) for a in df2[2]]

In [20]:
df2.columns = ['Rating', 'Company', 'Taxes']

In [21]:
df2.head()

,Rating,Company,Taxes
0,1,[ԳԱԶՊՐՈՄ ԱՐՄԵՆԻԱ],"13,570,605 41.2%"
0,2,[ԳՐԱՆԴ ՏՈԲԱԿՈ],"12,830,020 11.68%"
0,3,[ԶԱՆԳԵԶՈՒՐԻ ՊՂՆՁԱՄՈԼԻԲԴԵՆԱՅԻՆ ԿՈՄԲԻՆԱՏ],"6,684,445 18.98%"
0,4,[ՀԱՅԱՍՏԱՆԻ ԷԼԵԿՏՐԱԿԱՆ ՑԱՆՑԵՐ],"4,544,427 106.26%"
0,5,[ԳԵՈՊՐՈՄԱՅՆԻՆԳ ԳՈԼԴ],"4,295,157 3.61%"


In [22]:
df2.Taxes = [int(re.findall('.*\s', t)[0].strip().replace(',', '')) for t in df2.Taxes]

In [23]:
df2.head()

,Rating,Company,Taxes
0,1,[ԳԱԶՊՐՈՄ ԱՐՄԵՆԻԱ],13570605
0,2,[ԳՐԱՆԴ ՏՈԲԱԿՈ],12830020
0,3,[ԶԱՆԳԵԶՈՒՐԻ ՊՂՆՁԱՄՈԼԻԲԴԵՆԱՅԻՆ ԿՈՄԲԻՆԱՏ],6684445
0,4,[ՀԱՅԱՍՏԱՆԻ ԷԼԵԿՏՐԱԿԱՆ ՑԱՆՑԵՐ],4544427
0,5,[ԳԵՈՊՐՈՄԱՅՆԻՆԳ ԳՈԼԴ],4295157


In [24]:
print('Average Tax:', df2.Taxes.mean(), 'AMD')

Average Tax: 809770.225 AMD
